In [1]:
import numpy as np
import pandas as pd
import sys
import os
from sklearn.datasets import fetch_covtype
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

sys.path.append(os.path.abspath(".."))
from common import common

In [2]:
base_path = common.base_path

In [3]:
def get_customer_df():
    config = {
        'TARGET_COLUMN': 'Segmentation',
        
        # List of numerical columns (these are to be standardized)
        'NUMERICAL_COLUMNS': [],
        # List of categorical columns (these are to be one hot encoded)
        'CATEGORICAL_COLUMNS': [],
        # List of ordinal columns (these are to be label encoded)
        'ORDINAL_COLUMNS': [],
    }
    target_column = config['TARGET_COLUMN']
    all_df = pd.read_csv(f'{base_path}/datasources/customer/train.csv')
    
    # Headers of column
    main_labels = all_df.columns
    
    print('Normal class: ', all_df[target_column].mode())
    return (all_df, main_labels, config)

In [4]:
def get_processed_customer_df():
    all_df, main_labels, config = get_cicids_df()
    # print('main_labels', main_labels)
    target_column = config['TARGET_COLUMN']

    # Drop columns
    all_df = all_df.drop(columns='Label')
    all_df = all_df.loc[:, ~all_df.columns.str.contains('^Unnamed')]
    
    # Label Encoder
    le, all_df = common.label_encode(all_df, [target_column])
    
    config['TARGET_DICT'] = {index: label for index, label in enumerate(le.classes_)}
    config['INV_TARGET_DICT'] = {v: k for k, v in config['TARGET_DICT'].items()}
    print('TARGET_DICT', config['TARGET_DICT'])
    
    config['NORMAL_TARGET'] = config['INV_TARGET_DICT']['Benign']
    print('NORMAL_TARGET', config['NORMAL_TARGET'])

    main_labels = all_df.columns
    print('main_labels', main_labels)
    
    return (all_df, main_labels, config)

In [5]:
df,m,c = get_customer_df()

Normal class:  0    D
Name: Segmentation, dtype: object


In [6]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [13]:
df.shape

(590540, 435)